# Read results processed by MintPy

**Copyright (c) 2021, Lei Yuan**

## all functions

In [1]:
import os
import h5py
import numpy as np
from osgeo import gdal, osr
from scipy.io import netcdf


def read_h5(fname, label):
    with h5py.File(fname, 'r') as f:
        atr = dict(f.attrs)
        data = np.asarray(f[(label)])
    return data, atr


def get_lon_lat(atr):
    X_FIRST = float(atr['X_FIRST'])
    Y_FIRST = float(atr['Y_FIRST'])
    X_STEP = float(atr['X_STEP'])
    Y_STEP = float(atr['Y_STEP'])
    W = int(atr['WIDTH'])
    L = int(atr['LENGTH'])
    Y_END = Y_FIRST + L * Y_STEP
    X_END = X_FIRST + W * X_STEP

    X = np.linspace(X_FIRST, X_END, W)
    Y = np.linspace(Y_FIRST, Y_END, L)

    return Y, X


def get_dates(ts_file):
    dates, _ = read_h5(ts_file, 'date')
    dates = [int(d) for d in dates]

    return dates


def read_vel(vel_file, mask_file=None, ds_factor=1, out_vel_file=None):
    vel, atr = read_h5(vel_file, 'velocity')

    lat, lon = get_lon_lat(atr)
    lons, lats = np.meshgrid(lon, lat)

    vel = vel[::ds_factor, ::ds_factor]
    lons = lons[::ds_factor, ::ds_factor]
    lats = lats[::ds_factor, ::ds_factor]

    if mask_file:
        mask, _ = read_h5(mask_file, 'mask')
        mask = mask[::ds_factor, ::ds_factor]

        lons = lons[mask]
        lats = lats[mask]
        vel = vel[mask]

    lons = lons.reshape((-1, 1))
    lats = lats.reshape((-1, 1))
    vel = vel.reshape((-1, 1))

    num = np.arange(vel.shape[0]).reshape((-1, 1))

    vel *= 1000

    print('max velocity : ', np.max(vel))
    print('min velocity : ', np.min(vel))
    print('number of points : ', vel.shape[0])

    out_vel = np.hstack((num, lons, lats, vel))

    if out_vel_file:
        print('Writing data to {}'.format(out_vel_file))
        np.savetxt(out_vel_file, out_vel, fmt='%4f')
        print('done.')
        
    return out_vel


def read_vel_ts(ts_file, vel_file, mask_file=None, ds_factor=1, out_vel_file=None, out_ts_file=None):
    dates = get_dates(ts_file)

    ts, _ = read_h5(ts_file, 'timeseries')
    ts *= 1000

    out_vel = read_vel(vel_file, mask_file, ds_factor, out_vel_file)
    
    if mask_file:
        mask, _ = read_h5(mask_file, 'mask')
        mask = mask[::ds_factor, ::ds_factor]
    
    out_ts1 = out_vel
    for i in range(len(dates)):
        disp = ts[i]
        if mask_file:
            disp = disp[::ds_factor, ::ds_factor]
            disp = disp[mask]
        out_ts1 = np.hstack((out_ts1, disp.reshape((-1, 1))))

    tmp = out_ts1[:, 4:]
    tmp = tmp - tmp[:, 0].reshape((-1, 1))
    tmp = np.hstack((out_vel, tmp))
    out_ts1 = tmp

    print('max cumulative displacement: ', np.max(out_ts1[:, -1]))
    print('min cumulative displacement: ', np.min(out_ts1[:, -1]))

    first_line = np.asarray([[-1, -1, -1, -1]])
    dates = np.asarray(dates)
    first_line = np.hstack((first_line, dates.reshape((1, -1))))

    out_ts = np.vstack((first_line, out_ts1))

    if out_ts_file:
        print('Writing data to {}'.format(out_ts_file))
        np.savetxt(out_ts_file, out_ts, fmt='%4f')
        print('done.')


def vel2geotiff(vel_file, out_file, mask_file=None):
    vel, atr = read_h5(vel_file, 'velocity')
    vel *= 1000

    min_lon = float(atr['X_FIRST'])
    max_lat = float(atr['Y_FIRST'])

    lon_step = float(atr['X_STEP'])
    lat_step = float(atr['Y_STEP'])

    width, nlines = vel.shape[1], vel.shape[0]

    if mask_file:
        mask, _ = read_h5(mask_file, 'mask')
        vel[mask == False] = np.nan

    driver = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(out_file, width, nlines, 1, gdal.GDT_Float32)

    dataset.SetGeoTransform([min_lon, lon_step, 0, max_lat, 0, lat_step])

    sr = osr.SpatialReference()
    sr.SetWellKnownGeogCS('WGS84')

    dataset.SetProjection(sr.ExportToWkt())
    dataset.GetRasterBand(1).WriteArray(vel)
    print('Writing data to {}.'.format(out_file))
    dataset.FlushCache()
    dataset = None
    print('done.')
    
    
def write_gmt_simple(lons, lats, z, fname, title='default', name='z', scale=1.0, offset=0, units='meters'):
    """Writes a simple GMT grd file with one array.
    This is based on the gdal2grd.py script found at:
        http://http://www.vso.cape.com/~nhv/files/python/gdal/gdal2grd.py

    Parameters: lons : 1D Array of lon values
                lats : 1D Array of lat values
                z : 2D slice to be saved
                fname : Output file name
    Kwargs:     title : Title for the grd file
                name : Name of the field in the grd file
                scale : Scale value in the grd file
                offset : Offset value in the grd file
    """

    fid = netcdf.netcdf_file(fname, 'w')

    # Create a dimension variable
    fid.createDimension('side', 2)
    fid.createDimension('xysize', np.prod(z.shape))

    # Range variables
    fid.createVariable('x_range', 'd', ('side',))
    fid.variables['x_range'].units = 'degrees'

    fid.createVariable('y_range', 'd', ('side',))
    fid.variables['y_range'].units = 'degrees'

    fid.createVariable('z_range', 'd', ('side',))
    fid.variables['z_range'].units = units

    # Spacing
    fid.createVariable('spacing', 'd', ('side',))
    fid.createVariable('dimension', 'i4', ('side',))

    fid.createVariable('z', 'f', ('xysize',))
    fid.variables['z'].long_name = name
    fid.variables['z'].scale_factor = scale
    fid.variables['z'].add_offset = offset
    fid.variables['z'].node_offset = 0

    fid.title = title
    fid.source = 'MintPy'

    # Filling in the actual data
    fid.variables['x_range'][0] = lons[0]
    fid.variables['x_range'][1] = lons[-1]
    fid.variables['spacing'][0] = lons[1]-lons[0]

    fid.variables['y_range'][0] = lats[0]
    fid.variables['y_range'][1] = lats[-1]
    fid.variables['spacing'][1] = lats[1]-lats[0]

    # Range
    fid.variables['z_range'][0] = np.nanmin(z)
    fid.variables['z_range'][1] = np.nanmax(z)

    fid.variables['dimension'][:] = z.shape[::-1]
    fid.variables['z'][:] = np.flipud(z).flatten()
    fid.close()
    

def ts2grd(ts_file, out_dir, mask_file=None):
    ts, atr = read_h5(ts_file, 'timeseries')

    dates = get_dates(ts_file)
    dates = [str(d) for d in dates]

    lats, lons = get_lon_lat(atr)
    
    if mask_file:
        mask, _ = read_h5(mask_file, 'mask')

    if not os.path.isdir(out_dir):
        os.mkdir(out_dir)
    
    for date in dates:
        disp = ts[dates.index(date), :, :]
        disp -= ts[0, :, :]
        disp *= 1000

        if mask_file:
            disp[mask == False] = np.nan

        fname_out = os.path.join(out_dir, date + '.grd')
        print('Writing data to {}'.format(fname_out))
        write_gmt_simple(lons, np.flipud(lats), np.flipud(disp), fname_out, title='default', name=atr['FILE_TYPE'], scale=1.0, offset=0, units='mm')


def vel2grd(vel_file, out_file, mask_file=None):
    vel, atr = read_h5(vel_file, 'velocity')
    
    vel *= 1000

    lats, lons = get_lon_lat(atr)
    
    if mask_file:
        mask, _ = read_h5(mask_file, 'mask')
        vel[mask == False] = np.nan
    
    out_file = os.path.abspath(out_file)
    if not out_file.endswith('.grd'):
        out_file += '.grd'
    
    print('Writing data to {}'.format(out_file))
    write_gmt_simple(lons, np.flipud(lats), np.flipud(vel), out_file, title='default', name=atr['FILE_TYPE'], scale=1.0, offset=0, units='mm/yr')
    
    
def load_ts(ts_file):
    content = np.loadtxt(ts_file, np.float64)

    dates = np.asarray(content[0, 4:], np.int64)
    ts = content[1:, :]

    return ts, dates

## read velocity

In [ ]:
os.chdir('/ly/MintPy-Process/geo')

vel_file = 'geo_velocity.h5'
mask_file = 'geo_maskTempCoh.h5'

_ = read_vel(vel_file, mask_file=mask_file, ds_factor=1, out_vel_file='vel.txt')

In [ ]:
!clip_vel_ts.py vel.txt v cut.kmz

In [ ]:
!make_kmz_vel.py vel_clip.txt vel_clip.kmz -v -100 100 -c jet

## velocity2geotiff

In [ ]:
os.chdir('/ly/MintPy-Process/geo')

vel_file = 'geo_velocity.h5'
mask_file = 'geo_maskTempCoh.h5'

vel2geotiff(vel_file, 'velocity.tif', mask_file=mask_file)

## velocity2grd

In [ ]:
os.chdir('/ly/MintPy-Process/geo')

vel_file = 'geo_velocity.h5'
mask_file = 'geo_maskTempCoh.h5'

vel2grd(vel_file, 'velocity.grd', mask_file=mask_file)

## read time-series and velocity

In [ ]:
os.chdir('/ly/MintPy-Process/geo')

ts_file = 'geo_timeseries_tropHgt_ramp_demErr.h5'
vel_file = 'geo_velocity.h5'
mask_file = 'geo_maskTempCoh.h5'

read_vel_ts(ts_file, vel_file, mask_file=mask_file, ds_factor=1, out_vel_file='vel.txt', out_ts_file='ts.txt')

In [ ]:
!clip_vel_ts.py ts.txt t cut.kmz

In [ ]:
!make_kmz_ts.py ts_clip.txt ts_clip.kmz -v -100 100 -c jet

## ts2grd

In [ ]:
os.chdir('/ly/MintPy-Process/geo')

ts_file = 'geo_timeseries_tropHgt_ramp_demErr.h5'

mask_file = 'geo_maskTempCoh.h5'

ts2grd(ts_file, './grd', mask_file=mask_file)

## plot time-series displacement

In [2]:
import matplotlib.pyplot as plt
import datetime


def plot_displacement(num_list, ts_data, date, aspect=0.2, figsize=(15, 7), y_lim=[-100, 100], fig_name=None):
    fig, ax = plt.subplots(figsize=figsize)

    ax.set_title('time series displacement', fontsize=40, fontweight='bold')
    ax.set_aspect(aspect)
    ax.minorticks_on()

    ax.set_xlabel('date', fontsize=30, labelpad=10)
    ax.xaxis.grid(True, which='major')
    ax.xaxis.set_tick_params(rotation=30, labelsize=15)
    ax.set_xmargin(0.02)

    ax.set_ylabel('displacrment (mm)', fontsize=30, labelpad=10)
    ax.set_ylim(y_lim[0], y_lim[1])
    ax.yaxis.grid(True, which='major')
    ax.yaxis.set_tick_params(rotation=0, labelsize=15)

    date_str = [str(d) for d in date]

    date = [datetime.datetime.strptime(d, '%Y%m%d') for d in date_str]

    for num in num_list:
        disp = ts_data[num + 1, 4:]
        ax.plot(date, disp, label=str(num), marker='o', linewidth=0, markersize=10)

    ax.set_xticks(date[::4])
    ax.set_xticklabels(date_str[::4])
    ax.set_yticks(list(range(y_lim[0], y_lim[1] + 20, 20)))
    ax.legend(loc='best', fontsize=20, ncol=2)
    fig.show()

    if not fig_name is None:
        fig.savefig(fig_name, dpi=200, bbox_inches='tight', facecolor='white')


In [ ]:
ts, dates = load_ts('ts.txt')

In [ ]:
num_list = [69445]

plot_displacement(num_list, ts, dates, aspect=0.6, figsize=(30, 15), y_lim=[-200, 50], fig_name=None)